In [32]:
from typing import List
import pandas as pd
import sys
sys.path.append("../")
sys.path.append("../src/")
from tqdm.notebook import tqdm
from src.engine.morph import Morph
from src.engine.utils import fts_text_processing_passage, embedding_text_processing_passage
from src.engine.embedding import Embedding

morph_model = Morph()

In [30]:
data = pd.read_parquet("../data/raw_data.pq")
data.rename(columns={col:"raw_" + "_".join(col.split()) for col in data.columns.values if col not in ["index orig", "link"]}, inplace=True)
data.rename(columns={"link": "video_url"}, inplace=True)

data["_id"] = data["video_url"]

In [31]:
data.head()

,index orig,video_url,raw_description,raw_title_shazam,raw_subtitle_shazam,raw_transcription_whisper_v3,raw_image_captioning,_id
0,5000,https://cdn-st.rutubelist.ru/media/67/86/ebba7...,"Sejak saat itu, senyumku jadi palsu-----------...",Here's Your Perfect,Jamie Miller,もう! 雉真のバカ!雉真のせいで勉強できない!せっかくしたのに雉真があんな態度だから思い出す...,an anime character with blue hair and blue eyes,https://cdn-st.rutubelist.ru/media/67/86/ebba7...
1,5001,https://cdn-st.rutubelist.ru/media/73/3b/9c1d6...,#красивыедевушки #азиатки #boobs #грудь,走路像只鸭(DJ咚鼓版),"黑暗萝莉, 孙大佛爷, 索菲亚, 沈子凡 & 社长姐姐",Субтитры сделал DimaTorzok,a woman with blue skin and dark hair,https://cdn-st.rutubelist.ru/media/73/3b/9c1d6...
2,5002,https://cdn-st.rutubelist.ru/media/90/62/ff4c5...,"#boobs , #bigass , #girls , #pussy , #еда , #г...",Shaman King (Russian Version),SHAMAN KING,"Шаман Субтитры сделал DimaTorzok Краду, краду...",a man with blue hair and a microphone,https://cdn-st.rutubelist.ru/media/90/62/ff4c5...
3,5003,https://cdn-st.rutubelist.ru/media/67/59/68236...,#образ #lookbook #показ #неделямоды #капсула #...,Talking to the Moon,Bruno Mars,Так правильно выбрать актуальное худио. Ставь...,a person wearing gloves and a white shirt,https://cdn-st.rutubelist.ru/media/67/59/68236...
4,5004,https://cdn-st.rutubelist.ru/media/c2/ab/3ff35...,#уходзакожей #уходзасобой #бьютирутина #бьюти ...,Me So Horny,The 2 Live Crew,ТОП 5 стайлингов из масс-маркета ТОП 5 стайли...,a tube of tat hair care product on a white ba...,https://cdn-st.rutubelist.ru/media/c2/ab/3ff35...


In [33]:
# for fts
full_text = []
text_hashtags = []
song_name = []
song_author = []
song_name_transliterated = []
song_author_transliterated = []
video_hashtags = []
audio_hashtags = []
audio_transcription = []
# for embedding
text_to_embedds = []

for _, row in tqdm(data.iterrows(),):
    text_to_embedd: str = embedding_text_processing_passage(
        morph=morph_model,
        raw_description=row.raw_description,
        raw_audio_transcription=row.raw_transcription_whisper_v3,
        raw_video_hashtags=row.raw_image_captioning,
    )

    # text_to_fts: Dict[str, str] = fts_text_processing_passage(
    #     morph=morph_model,
    #     raw_description=row.raw_description,
    #     raw_song_name=row.raw_title_shazam,
    #     raw_song_author=row.raw_subtitle_shazam,
    #     raw_song_name_transliterated=...,
    #     raw_song_author_transliterated=...,
    #     raw_audio_transcription=row.raw_transcription_whisper_v3,
    #     raw_video_hashtags=row.raw_image_captioning,
    # )

    text_to_embedds.append(text_to_embedd)
    # full_text.append(text_to_fts["full_text"])
    # text_hashtags.append(text_to_fts["clean_description"])
    # song_name.append(text_to_fts["clean_song_name"])
    # song_author.append(text_to_fts["clean_song_author"])
    # song_name_transliterated.append(text_to_fts["clean_song_name_transliterated"])
    # song_author_transliterated.append(text_to_fts["clean_song_author_transliterated"])
    # video_hashtags.append(text_to_fts["clean_video_hashtags"])
    # audio_hashtags.append(text_to_fts["clean_audio_hashtags"])
    # audio_transcription.append(text_to_fts["clean_audio_transcription"])

0it [00:00, ?it/s]

In [37]:
data["text_to_embedd"] = text_to_embedds
data["full_text"] = full_text
data["text_hashtags"] = text_hashtags
data["song_name"] = song_name
data["song_author"] = song_author
data["song_name_transliterated"] = song_name_transliterated
data["song_author_transliterated"] = song_author_transliterated
data["video_hashtags"] = video_hashtags
data["audio_hashtags"] = audio_hashtags
data["audio_transcription"] = audio_transcription

SyntaxError: unexpected EOF while parsing (572476986.py, line 1)

In [ ]:
embedding = Embedding(device="cuda:0")

bs = 500
text_embeddings: List[List[float]] = []
for i in tqdm(range(0, len(text_to_embedds), bs)):
    batch = text_to_embedds[i:i+bs]
    text_embeddings.extend(embedding(texts=batch))

In [ ]:
data["embedding"] = text_embeddings

In [ ]:
data.to_parquet("../data/final.pq")